In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':2048,
    'SEED':41
}

PATH = os.getcwd() + '/data/'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
train_data = pd.read_csv(PATH + 'train.csv')
submit = pd.read_csv(PATH + 'sample_submission.csv')
display(train_data.head(3))
submit.shape

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(15890, 22)

## CNN-LSTM MODEL
![스크린샷 2023-08-04 오후 5 16 00](https://github.com/Megvii-BaseDetection/YOLOX/assets/103639510/6e612140-39b5-4198-bb14-81ff2aca5f22)


In [ ]:
class Conv1d_LSTM(nn.Module):
    def __init__(self, in_channel=5, hidden_size = 128, out_channel=CFG['PREDICT_SIZE']):
        super(Conv1d_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.conv1d_1 = nn.Conv1d(in_channels=in_channel,
                                out_channels=16,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.conv1d_2 = nn.Conv1d(in_channels=16,
                                out_channels=32,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.lstm = nn.LSTM(input_size = 32,
                            hidden_size=hidden_size,
                            num_layers=1,
                            bias=True,
                            bidirectional=False,
                            batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, out_channel)
        )

        self.actv = nn.ReLU()
        # self.dense1 = nn.Linear(hidden_size, hidden_size//2)
        # self.dropout = nn.Dropout()
        # self.dense2 = nn.Linear(hidden_size//2, out_channel)
    
    def init_hidden(self, batch_size, device):
        return (
            torch.zeros(1, batch_size, self.hidden_size, device = device),
            torch.zeros(1, batch_size, self.hidden_size, device = device)
        )

    def forward(self, x):
	# Raw x shape : (B, TRAIN_WINDOW_SIZE, in_channel) => (B, 90, 5)
        
        # Shape : (B, F: in_channel, S: TRAIN_WINDOW_SIZE) => (B, 5, 90)
        x = x.transpose(1, 2)
        # Shape with Conv1d_1 : (B, F, S) == (B, C, S) || C = channel => (B, 16, 90)
        x = self.conv1d_1(x)
        ## Shape with Conv1d_2 : (B, C, S) => (B, 32, 90)
        x = self.conv1d_2(x)
        ## Reshape : (B, S, C) == (B, S, F) => (B, 90, 32)
        x = x.transpose(1, 2)
        
        # self.lstm.flatten_parameters()
        # # Shape : (B, S, H) // H = hidden_size => (B, 90, 512)
        # _, (hidden, _) = self.lstm(x)
        # # Shape : (B, H) // -1 means the last sequence => (B, 512)
        # x = hidden[-1]
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM Layer
        lstm_out, hidden = self.lstm(x, hidden)

        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))
        x = output.squeeze(1)
        
        # # Shape : (B, 512)
        # x = self.dense1(x)
        # # ReLU
        # x = F.relu(x)
        # # Shape : (B, H) => (B, 256)
        # x = self.dropout(x)
        # # Shape : (B, O) // O = output => (B, 21)
        # x = self.dense2(x)

        return x

## Make Dataset

In [10]:
LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'
train_gate = pd.read_csv(LOADPATH + 'train_gate_before_scaling.csv')
train_gate.head(3)

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,1,6,37,0,0.108580,0.118135,0.199903,0.310480,...,0.035216,0.044758,0.041279,0.063038,0.000000,0.029457,0.033945,0.023542,0.035053,0.056512
1,1,B002-00002-00001,2,7,43,1,0.109679,0.255403,0.164446,0.107420,...,0.077546,0.110759,0.085193,0.136872,0.252020,0.178075,0.070021,0.074956,0.205545,0.097419
2,2,B002-00002-00002,2,7,43,1,0.114349,0.248839,0.166527,0.106789,...,0.081300,0.105084,0.084713,0.081488,0.082202,0.066784,0.066833,0.080457,0.084743,0.094155


In [11]:
train_size = CFG['TRAIN_WINDOW_SIZE'] # 90days
data = train_gate.iloc[:, 2:].copy()

'''
평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
data : 일별 판매량
train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
'''
num_rows = len(data) # 15682개

input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1), dtype = np.float16)

for i in tqdm(range(num_rows)):
    encode_info = np.array(data.iloc[i, :4])
    sales_data = np.array(data.iloc[i, -train_size:])
        
    window = sales_data[-train_size : ]
    temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
    input_data[i] = temp_data


  0%|          | 0/15682 [00:00<?, ?it/s]

In [ ]:
# # Label Encoding
# label_encoder = LabelEncoder()
# categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

# for col in categorical_columns:
#     label_encoder.fit(train_data[col])
#     train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간 => 90 Days
    predict_size : 추론할 기간 => 21 Days
    '''
    num_rows = len(data) # 15890
    window_size = train_size + predict_size # 90 + 21 = 111
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1), dtype = np.float16)
    # (5609170, 90, 5)
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size), dtype = np.float16)
    # (5640950, 21)

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Label: 대분류, 중분류, 소분류, 브랜드
        sales_data = np.array(data.iloc[i, 4:]) # 날짜 데이터: 2022-01-01 ~ 2023-04-04
        
        for j in range(len(sales_data) - window_size + 1): # 0 ~ 348
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']): #90
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1), dtype = np.float16)
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [ ]:
class IndividualMinMaxScaler:
    def __get_min_val(self, df, date_cols):
        df_mins = df[date_cols].min(axis=1).to_numpy()
        return df_mins

    def __get_max_val(self, df, date_cols):
        df_maxs = df[date_cols].max(axis=1).to_numpy()
        return df_maxs


    def fit(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns  # 일자 변수들만 Scale하기
                     if col.startswith("2")]

        self.min_val = self.__get_min_val(df, date_cols)
        self.max_val = self.__get_max_val(df, date_cols)
        denom = self.max_val - self.min_val
        self.denom = np.where(denom==0, 1, denom)

    def transform(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns
                     if col.startswith("2")]
        return df[date_cols] \
            .apply(lambda x: (x-self.min_val)/self.denom)

    def fit_transform(self, df: pd.DataFrame):
        self.fit(df)
        return self.transform(df)

In [ ]:
train_scale = train_gate.copy()

train_scale.drop(['ID', '제품'], axis = 1, inplace = True)
scaler = IndividualMinMaxScaler()
train_scale.iloc[:, 4:] = scaler.fit_transform(train_gate)
print(min(train_scale.iloc[:, 6]), max(train_scale.iloc[:, 6]))
train_scale.head(3)

In [ ]:
train_input, train_target = make_train_data(train_scale)
test_input = make_predict_data(train_scale)

In [ ]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

print(train_input.shape)
print(train_target.shape)
print(val_input.shape)
print(val_target.shape)
print(test_input.shape)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Loss

#### Metric

In [ ]:
def pseudo_sfa(pred, df):
    pred_length = pred.shape[1] - 1
    true = df.iloc[:, -pred_length:].reset_index() \
        .rename(columns={"index": "ID"})

    main_id = {}
    for main_cat in df["대분류"].unique():
        main_id[main_cat] = df.query("대분류==@main_cat")["ID"].to_list()

    psfa = []
    for main_cat in main_id.keys():
        indices = true["ID"].isin(main_id[main_cat])

        true_arr = true[indices].iloc[:, 1:].to_numpy()
        pred_arr = pred[indices].iloc[:, 1:].to_numpy()

        eps = np.ones((true_arr.shape)) / 1e8

        true_sum = true_arr.sum(axis=0)
        true_sum = np.stack([true_sum]*len(true_arr)) + eps
        true_rate = true_arr / true_sum

        abs_error = np.abs(true_arr - pred_arr)
        denom = np.maximum(true_arr, pred_arr+eps)
        
        score = 1 - (1 / true_arr.shape[1]
                     * (abs_error / denom) * true_rate).sum()
        psfa.append(score)
        print(main_cat, score)

    return np.mean(psfa)

#### Loss Function

In [ ]:
class PsfaLoss(nn.Module):
    def __init__(self, scaler, df):
        super().__init__()
        self.scaler = scaler
        self.main_cats = df.groupby("대분류")["ID"].unique().values
    
    def forward(self, pred, true):
        # pred: [batch_size, length, products(15890)]
        pred = pred * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)
        true = true * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)

        L1scaled = torch.abs(true-pred) / torch.maximum(pred, true+1e-8)
        
        rate = torch.zeros_like(true)
        for i in range(len(self.main_cats)):
            rate[:, :, self.main_cats[i]] = \
                true[:, :, self.main_cats[i]] \
                / (true[:, :, self.main_cats[i]].sum(dim=-1, keepdim=True) + 1e-8) \
                / len(self.main_cats)
        return (L1scaled * rate).sum() / (true.shape[0] * true.shape[1])

In [ ]:
scaler = IndividualMinMaxScaler()
loss = PsfaLoss(scaler, train_gate)

## Train Function

In [ ]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [ ]:
device

In [ ]:
model = Conv1d_LSTM()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
infer_model, train_loss, val_loss = train(model, optimizer, train_loader, val_loader, device)

In [ ]:
SAVE_PATH = os.getcwd() + '/cnn_lstm_with_gate_data.pth'
torch.save(infer_model.state_dict(), SAVE_PATH)

In [ ]:
infer_model.state_dict()

In [ ]:
train_loss = np.array(train_loss)
val_loss = np.array(val_loss)

In [ ]:
np.save(PATH + 'train_loss', train_loss)
np.save(PATH + 'val_loss', val_loss)

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
prediction = inference(infer_model, test_loader, device)
prediction.shape

In [ ]:
train_gate.ID

In [ ]:
submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction

In [ ]:
SAVE_SUBMIT = os.getcwd() + '/data/cnn_lstm_test_2.csv'
submit.to_csv(SAVE_SUBMIT)

In [ ]:
submit